# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/cris/anaconda3/bin/python (found version "3.7.3") 
-- Found PythonLibs: /home/cris/anaconda3/lib/libpython3.7m.so
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/cris/Documents/Facultad/Met Num 2c_2019/TP2/metnum-tp2-20192c
-- Con

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment


/home/cris/Documents/Facultad/Met Num 2c_2019/TP2/metnum-tp2-20192c/notebooks
Python 3.7.3


In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

tar: *.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Cantidad de documentos: 12500


In [4]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,This show comes up with interesting locations ...,neg,5100_4.txt
freq,6275,2,6322,2


In [5]:
def get_instances(df, train_count, test_count):
    # particiono cantidad de instancias de test y train
    total_train = df[df.type == 'train']
    total_test = df[df.type == 'test']

    train = total_train.sample(n=train_count,random_state=1)
    test = total_test.sample(n=test_count,random_state=1)
    
    return train, test



In [6]:
train, test = get_instances(df, 100, 100)

text_train = train["review"]
label_train = train["label"]

text_test = test["review"]
label_test = test["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 100
Cantidad de instancias de test = 100


In [7]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.52 pos 0.48 neg


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
def vectorize(text_train, label_train, text_test, label_test, max_features):
    vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=max_features)
    
    vectorizer.fit(text_train)

    X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
    X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values
    
    return X_train, y_train, X_test, y_test

In [9]:
X_train, y_train, X_test, y_test = vectorize(text_train, label_train, text_test, label_test, 100)

In [10]:
import sentiment

clf = sentiment.KNNClassifier(10)

clf.fit(X_train, y_train)


In [11]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.54
CPU times: user 42.3 ms, sys: 0 ns, total: 42.3 ms
Wall time: 42 ms


In [13]:
# que empiecen los test
import sentiment

max_features_values = [1000, 1500]
train_values = [1000, 2000, 3000, 4000]
test_values = [1000]
k_values = [250,500, 750, 1000, 1250, 1500, 1750, 2000]

#sin pca

for ks in k_values:
    clf = sentiment.KNNClassifier(ks)
    for train_v in train_values:
        if ks > train_v:
            continue
        for test_v in test_values:
            train, test = get_instances(df, train_v, test_v)
            
            text_train = train["review"]
            label_train = train["label"]

            text_test = test["review"]
            label_test = test["label"]
            
            for max_feat in max_features_values:
                X_train, y_train, X_test, y_test = vectorize(text_train, label_train, text_test, label_test, max_feat)
                
                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                acc = accuracy_score(y_test, y_pred)
                
                with open("..//data//out_{}//out_{}_{}_{}_{}.txt".format(max_feat,max_feat,ks,train_v,test_v), "w") as f:
                    f.write("k: {}\n".format(ks))
                    f.write("max_features: {}\n".format(max_feat))
                    f.write("train: {}\n".format(len(text_train)))
                    f.write("test: {}\n".format(len(text_test)))
                    f.write("Accuracy: {}\n".format(acc))
            
